# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md) before you start.

# Imports

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops
from object_detection.builders import model_builder
from google.protobuf import text_format
import tensorflow as tf

if StrictVersion(tf.__version__) < StrictVersion('1.9.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.9.* or later!')


/home/wc-gpu/MasterThesis/models/research/object_detection/utils/visualization_utils.py:27: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/home/wc-gpu/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/wc-gpu/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/wc-gpu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/wc-gpu/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/wc-gpu/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line

## Env setup

In [2]:
# This is needed to display the images.
%matplotlib inline

## Object detection imports
Here are the imports from the object detection module.

In [3]:
from utils import label_map_util

from utils import visualization_utils as vis_util

/home/wc-gpu/MasterThesis/models/research/object_detection/utils/visualization_utils.py:27: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/home/wc-gpu/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/wc-gpu/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/wc-gpu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/wc-gpu/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/wc-gpu/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_FROZEN_GRAPH` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [4]:
# What model to download.
# MODEL_NAME = 'prod_faster_rcnn_resnet50_coco'
MODEL_NAME = 'faster_rcnn_resnet50_coco_2018_01_28'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

## Download Model

In [5]:
# opener = urllib.request.URLopener()
# opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
# tar_file = tarfile.open(MODEL_FILE)
# for file in tar_file.getmembers():
#   file_name = os.path.basename(file.name)
#   if 'frozen_inference_graph.pb' in file_name:
#     tar_file.extract(file, os.getcwd())

## Load a (frozen) Tensorflow model into memory.

In [6]:
model_config="./prod_faster_rcnn_resnet50_coco/model.config"

In [7]:
from object_detection.protos import model_pb2
model_proto = model_pb2.DetectionModel()
with open(model_config, 'r') as myfile:
    config_string=myfile.read()
    text_format.Merge(config_string, model_proto)



In [8]:
detection_model = model_builder.build(model_config=model_proto,
                                      is_training=False)

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [9]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

## Helper code

In [10]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Extract proposal features

In [11]:
def extract_region_proposal_features(image, model):
    tf.reset_default_graph()
   
    tf_image = tf.convert_to_tensor(np.expand_dims(image, 0), tf.float32)
    resized_image, true_image_shape = detection_model.preprocess(tf_image)
    rpn_features_to_crop, rett = model._feature_extractor.extract_proposal_features(
    resized_image, scope=detection_model.first_stage_feature_extractor_scope)
    init_op = tf.global_variables_initializer()

    with tf.Session() as sess:
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coord)
        sess.run(init_op)
        rpn_features_to_crop_out = sess.run(rpn_features_to_crop)
#         print (rpn_features_to_crop_out.shape, type(rpn_features_to_crop_out))
        coord.request_stop()
        coord.join(threads)
        return rpn_features_to_crop_out

In [12]:
# SESSIONS_PATH = "/Users/mbaltac/Home/MasterThesis/MasterThesis/labeled_images_shopping_sessions/labeled"
SESSIONS_PATH = "/home/wc-gpu/MasterThesis/session_data"

In [20]:
features = None
for session in sorted(os.listdir(SESSIONS_PATH), reverse=True):
    try:
        a = int(session)
    except:
        print (session)
        continue
    
    session_path = os.path.join(SESSIONS_PATH, session)
    for image_fname in sorted(os.listdir(session_path)):
        features_path = os.path.join(session_path, "features")
        if not os.path.exists(os.path.join(features_path, image_fname+".feature.coco")):
            image_path = os.path.join(session_path, image_fname)
            if not image_path.endswith(".jpg"):
                continue

            image = Image.open(image_path)
            # the array based representation of the image will be used later in order to prepare the
            # result image with boxes and labels on it.
            image_np = load_image_into_numpy_array(image)
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            # Actual detection.

            features = extract_region_proposal_features(image_np, detection_model)

            features_path = os.path.join(session_path, "features")
            if not os.path.exists(features_path):
                os.makedirs(features_path)

            with open(os.path.join(features_path, image_fname+".feature.coco"), "wb") as f:
                np.save(f, features)
        else:
            print("skipped", os.path.join(features_path, image_fname+".feature.coco"))

#         slc = features[0][:,:,500]
#         slc *= 255.0/slc.max()  

#         img = Image.fromarray(slc)
#         img = img.convert('RGB')

#         plt.figure(figsize=IMAGE_SIZE)
#         plt.imshow(image)

#         plt.figure(figsize=IMAGE_SIZE)
#         plt.imshow(img)

#         plt.figure(figsize=IMAGE_SIZE)
#         plt.imshow(image_np)
    

dataset.pkl
skipped /home/wc-gpu/MasterThesis/session_data/123429/features/ecenter_porta_123429_1530721325.332921_TOP_FRONT_EventResolved.jpg.feature.coco
skipped /home/wc-gpu/MasterThesis/session_data/123429/features/ecenter_porta_123429_1530721347.737312_TOP_FRONT_EventResolved.jpg.feature.coco
skipped /home/wc-gpu/MasterThesis/session_data/123429/features/ecenter_porta_123429_1530721348.776711_TOP_FRONT_EventResolved.jpg.feature.coco
skipped /home/wc-gpu/MasterThesis/session_data/123429/features/ecenter_porta_123429_1530721377.785235_TOP_FRONT_EventResolved.jpg.feature.coco
skipped /home/wc-gpu/MasterThesis/session_data/123429/features/ecenter_porta_123429_1530721407.979311_TOP_FRONT_EventResolved.jpg.feature.coco
skipped /home/wc-gpu/MasterThesis/session_data/123429/features/ecenter_porta_123429_1530721409.369466_TOP_FRONT_EventResolved.jpg.feature.coco
skipped /home/wc-gpu/MasterThesis/session_data/123429/features/ecenter_porta_123429_1530721411.242173_TOP_FRONT_EventResolved.jpg.

INFO:tensorflow:Scale of 0 disables regularizer.


I0110 22:01:22.214964 139750346217216 tf_logging.py:115] Scale of 0 disables regularizer.


INFO:tensorflow:Scale of 0 disables regularizer.


I0110 22:01:24.680183 139750346217216 tf_logging.py:115] Scale of 0 disables regularizer.


...

I0110 23:28:58.241453 139750346217216 tf_logging.py:115] Scale of 0 disables regularizer.


INFO:tensorflow:Scale of 0 disables regularizer.


I0110 23:29:00.626297 139750346217216 tf_logging.py:115] Scale of 0 disables regularizer.


INFO:tensorflow:Scale of 0 disables regularizer.


I0110 23:29:03.059899 139750346217216 tf_logging.py:115] Scale of 0 disables regularizer.


In [ ]:
def one_detection_features(features, xmin, ymin, xmax, ymax):
    final_feature = np.zeros(1024)
    xmin = xmin*features.shape[1]
    ymin = xmin*features.shape[0]
    xmax = xmin*features.shape[1]
    ymax = xmin*features.shape[0]
    for i in range(1024):
        final_feature[i] = np.max(features[0][ymin:ymax,xmin:xmax,i])
    
    return final_feature

In [14]:
features[0].shape

(38, 50, 1024)

In [15]:
slc = features[0][:10,:,500]

In [16]:
final_feature = np.zeros(1024)

In [17]:
for i in range(1024):
    final_feature[i] = np.max(features[0][:,:,i])

In [ ]:
final_feature

In [ ]:
max(final_feature)